In [1]:
!pip install transformers datasets seqeval
!pip install torch 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=6b52054b26f6ca4c371f4f05eb674cdbe4419e2aa180cc2aec486a6204d98d69
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver 

In [3]:
from datasets import load_dataset
dataset = load_dataset('conll2003', split='train')

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
unique_labels = set(label for sublist in dataset['labels'] for label in sublist)
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

In [38]:
import os
print(os.getcwd())
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaTokenizerFast
from datasets import Dataset, Features, Sequence, Value
from transformers import TrainingArguments
from transformers import XLMRobertaForTokenClassification, AutoModelForTokenClassification, AutoTokenizer, Trainer

/content


In [41]:

def load_conll_text(file_path):
    sentences = []
    labels = []
    with open(file_path, 'r', encoding='utf-8') as f:
        sentence = []
        label = []
        for line in f:
            if line.strip():  # Non-empty line
                token, label_item = line.split()
                sentence.append(token)
                label.append(label_item)
            else:  # Empty line indicates end of a sentence
                sentences.append(sentence)
                labels.append(label)
                sentence = []
                label = []
    return pd.DataFrame({'tokens': sentences, 'labels': labels})

file_path = r'labeled_data_conll.txt'
# Load your CoNLL file
df = load_conll_text(file_path)

In [42]:

df.head()

,tokens,labels
0,"[3, እስከ, 260, ሙቀት, መቆቆም, የሚችል, ዋጋ550ብር, አድራሻ, ...","[O, O, O, O, O, O, I-PRICE, O, O, O, O, O, O, ..."
1,"[ጊዜ, ቆጣቢ, ስላይስ, ማድረጊያ, ለእጅ, ሴፍቲ, ተመራጭ, ለድንች, ለ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[2, 1, 1, ዋጋ, 2, 500, ብር, ውስን, ፍሬ, ነው, ያለው, አድ...","[O, O, O, B-PRICE, I-PRICE, I-PRICE, I-PRICE, ..."
3,"[2, 1, 1, ዋጋ, 2, 500, ብር, ውስን, ፍሬ, ነው, ያለው, አድ...","[O, O, O, B-PRICE, I-PRICE, I-PRICE, I-PRICE, ..."
4,"[31, ዋጋ3000ብር, ውስን, ፍሬ, ነው, ያለው, አድራሻ, ቁ1, ስሪ,...","[O, I-PRICE, O, O, O, O, O, O, O, O, O, O, O, ..."


In [43]:
unique_labels = set(label for sublist in df['labels'] for label in sublist)
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

In [44]:
unique_labels

{'B-LOC', 'B-PRICE', 'B-PROD', 'I-PRICE', 'O'}

In [45]:
df['labels'] = df['labels'].apply(lambda x: [label2id[label] for label in x])

In [46]:

features = Features({
    'tokens': Sequence(Value('string')),  # List of strings for tokens
    'labels': Sequence(Value('int32'))    # List of integers for labels
})

In [47]:
# Convert DataFrame to Hugging Face Dataset with specified features
dataset = Dataset.from_pandas(df[['tokens', 'labels']], features=features)

In [48]:
# Explore the datast
dataset

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 3009
})

In [49]:
# Initialize the Fast Tokenizer
# Use the fast tokenizer
# For XLM-Roberta
tokenizer = XLMRobertaTokenizerFast.from_pretrained(
    "xlm-roberta-base",
    clean_up_tokenization_spaces=True
    )


In [50]:
# Tokenization and alignment function
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True, padding="max_length", max_length=128)  # Set max_length as needed
  labels = []

  for i in range(len(examples['tokens'])):
      label = examples['labels'][i]
      tokenized_label = [-100] * len(tokenized_inputs['input_ids'][i])  # Default label for all tokens

      # Aligning labels with tokens
      for j, token in enumerate(tokenized_inputs['input_ids'][i]):
          # Check if this token corresponds to the original word
          original_word_idx = tokenizer.decode(token).strip()
          if original_word_idx in examples['tokens'][i]:
              token_index = examples['tokens'][i].index(original_word_idx)
              tokenized_label[j] = label[token_index]  # Use the corresponding label

      labels.append(tokenized_label)

  tokenized_inputs['labels'] = labels
  return tokenized_inputs

In [51]:
# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3009 [00:00<?, ? examples/s]

In [52]:
tokenized_dataset

Dataset({
    features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 3009
})

In [53]:
# Split into train and validation datasets
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)  # 90% train, 10% validation

In [54]:
# Print the lengths of input_ids, attention_mask, and labels for verification
print(f"Number of samples: {len(tokenized_dataset)}")
print(f"Input IDs length: {[len(x) for x in tokenized_dataset['input_ids']]}")
print(f"Attention Mask length: {[len(x) for x in tokenized_dataset['attention_mask']]}")
print(f"Labels length: {[len(x) for x in tokenized_dataset['labels']]}")

Number of samples: 3009
Input IDs length: [128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 12

In [55]:
# Check the train and test split
train_test_split

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 2708
    })
    test: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 301
    })
})

In [56]:
# Set up training arguments with adjustments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",     # Evaluates at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    num_train_epochs=3,
    weight_decay=0.01,               # Strength of weight decay
    max_grad_norm=1.0,  # Gradient clipping
    logging_dir='./logs',            # Directory for storing logs
    logging_strategy="steps",        # Log at regular intervals
    logging_steps=50,                # Log every 50 steps
    save_strategy="epoch",           # Save model at the end of each epoch
    report_to="none",                # Only show logs in the output (no TensorBoard)
)

In [57]:
# Initialize each of the models
# For XLM-Roberta
model_xlmr = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(unique_labels)) # Ensure unique_labels is defined

# For DistilBERT
model_distilbert = AutoModelForTokenClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=len(unique_labels))

# For mBERT
model_distilbert = AutoModelForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(unique_labels))

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
trainer_xlmr = Trainer(
    model=model_xlmr,
    args=training_args,
    train_dataset=train_test_split['train'],
    eval_dataset=train_test_split['test'],  # Changed from validation to test based on split
)
trainer_distilbert = Trainer(
    model=model_distilbert,
    args=training_args,
    train_dataset=train_test_split['train'],
    eval_dataset=train_test_split['test'],  # Changed from validation to test based on split
)
trainer_mbert = Trainer(
    model=model_distilbert,
    args=training_args,
    train_dataset=train_test_split['train'],
    eval_dataset=train_test_split['test'],  # Changed from validation to test based on split
)

In [60]:
# Save the model
model.save_pretrained("./fine_tuned_ner_model")
tokenizer.save_pretrained("./fine_tuned_ner_model")

('./fine_tuned_ner_model/tokenizer_config.json',
 './fine_tuned_ner_model/special_tokens_map.json',
 './fine_tuned_ner_model/sentencepiece.bpe.model',
 './fine_tuned_ner_model/added_tokens.json',
 './fine_tuned_ner_model/tokenizer.json')

In [ ]:
!pip install shap lime

In [ ]:
import shap
explainer = shap.Explainer(model, tokenized_train_dataset)
shap_values = explainer(tokenized_eval_dataset)
shap.plots.text(shap_values[0])

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer()
explanation = explainer.explain_instance("ምርት ዋጋ በአዲስ አበባ", model.predict_proba, num_features=6)
explanation.show_in_notebook()

In [39]:
try:
    # Check if 'trainer' is defined
    if 'trainer' in locals() or 'trainer' in globals():
        print("The 'trainer' variable is defined.")

        # Proceed with evaluation
        eval_results = trainer.evaluate()
        print(eval_results)
    else:
        print("The 'trainer' variable is NOT defined. Please run the cell where it is defined first.")

except NameError:
    print("The 'trainer' variable is NOT defined. Please run the cell where it is defined first.")

The 'trainer' variable is NOT defined. Please run the cell where it is defined first.
